In [ ]:
!pip install snscrape transformers

In [2]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
from tqdm.notebook import tqdm
#from google.colab import files
from transformers import pipeline
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [55]:
#stock_list = ['AAPL', 'AFG', 'AMZN', 'BAC', 'EBAY', 'META', 'MSFT', 'NVDA', 'ORCL', 'ROKU', 'TSLA']
stock = 'AAPL'
tweets_list = []
for i, tweet in tqdm(enumerate(sntwitter.TwitterSearchScraper(f"${stock} since:2021-01-04 until:2022-11-10").get_items()), total=10000,):
    if i > 10000:
        break
    tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])  # declare the attributes to be returned
tweet_df = pd.DataFrame(tweets_list, columns=["Datetime", "Tweet Id", "Text", "Username"])#to df

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
#Transform
fileName = 'Twitter'+stock+'.xlsx'
tweet_df['Datetime'] = pd.to_datetime(tweet_df['Datetime'])
tweet_df['Datetime'] = tweet_df['Datetime'].dt.tz_localize(None)
tweet_df.replace('|', ',')
tweet_df.sort_values(by = 'Datetime', ascending=False, na_position='first', inplace = True)
tweet_df.info()

In [ ]:
#Sentiment: Loading model
model = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model)
sentiment_task("Covid cases are increasing fast!")

In [ ]:
#applying sentiment to dataframe
sent_results = {}
count = 0
for i, d in tqdm(tweet_df.iterrows(), total=len(tweet_df)):
    sent = sentiment_task(d["Text"])
    sent_results[d["Tweet Id"]] = sent
    count += 1
    if count == 10000:
        break

In [41]:
#Transformations
sent_df = pd.DataFrame(sent_results).T
sent_df["label"] = sent_df[0].apply(lambda x: x["label"])
sent_df["score"] = sent_df[0].apply(lambda x: x["score"])
sent_df = sent_df.merge(tweet_df.set_index("Tweet Id"), left_index=True, right_index=True)
sent_df.drop(sent_df.columns[0], axis = 1, inplace = True)
sent_df["ticker"] = stock

In [52]:
#save from colab to local
writer = pd.ExcelWriter(fileName)
sent_df.to_excel(writer, sheet_name = 'sentiment', index=False)
writer.save()
#download
files.download(fileName)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>